Documentation:

This part creates a dataset for generating the candidates of users request,
it takes:[{column:query}, {coulumn, query}], and will fetch the top 5 results


the column list accepted here is :

* 1.rate             
* 2.approx_cost  
the numerial variable can take one number, two numbers, or a number and a direction

for example: [
  {'rate',(3,4)},{'approx_cost',200}]

----------------

* 3.location         
* 4.rest_type        
* 5.dish_liked      
* 6.cuisines         
* 7.menu_item        
* 8.listed_type    

pass the variable and the query:

for example
[{'cuisines': 'Italian'},
{'location': 'Banashankari'}]



## Data proprecessing


In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')
file_path ="/content/drive/My Drive/Colab Notebooks/data/zomato.csv"
zomato = pd.read_csv(file_path)

Mounted at /content/drive


In [4]:
zomato.rename(columns={'approx_cost(for two people)':'approx_cost','listed_in(type)':'listed_type','listed_in(city)':'listed_city'},inplace=True)

In [5]:
#1. process the data type for float variables :convert str to float

# Define a lambda function to remove the '/5' from the rate column and convert to float
remove_slash = lambda x: x.replace('/5', '') if isinstance(x, str) else x

# Apply the lambda function to the rate column
# Ensure the column is treated as string before calling .replace()
zomato['rate'] = zomato['rate'].astype(str).apply(remove_slash).str.strip()

# Now, safely convert the rate column to float
zomato['rate'] = pd.to_numeric(zomato['rate'], errors='coerce')


#convert cost into float

#zomato['approx_cost'] = zomato['approx_cost'].astype(str).replace(',', '').astype(float, errors='ignore')
zomato['votes'] = zomato['votes'].astype(str).str.strip().astype(float, errors='ignore')
zomato['approx_cost'] = zomato['approx_cost'].astype(str)
zomato['approx_cost'] = zomato['approx_cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['approx_cost'] = zomato['approx_cost'].astype(float)


In [6]:
#2. calculated a weighted rating score based on voting

# Calculate C as the mean rating across all restaurants
C = zomato['rate'].mean()

# Set m to a value you choose (for example, the 90th percentile of the votes)
m = zomato['votes'].quantile(0.90)

# Define a function to compute the weighted rating for each row
def weighted_rating(x, m, C):
    v = x['votes']
    R = x['rate']
    return (v/(v+m) * R) + (m/(m+v) * C)

# Apply the function to your DataFrame
zomato['weighted_rating'] = zomato.apply(weighted_rating, axis=1, args=(m, C))

In [7]:
zomato.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost,reviews_list,menu_item,listed_type,listed_city,weighted_rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775.0,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,3.909956
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1,787.0,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,3.911487
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8,918.0,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.756826
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7,88.0,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.700399
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8,166.0,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.719465


In [12]:

zomato.drop_duplicates(subset=['name','address'],inplace=True)

In [14]:
zomato.drop(['url','address','phone'],axis=1,inplace=True)

In [15]:
zomato.to_csv('/content/drive/My Drive/Colab Notebooks/data/zomato_ready.csv', index=False)

In [ ]:
zomato['weighted_rating'].sort_values(ascending=False)[0:5]

49627    4.851908
49170    4.851908
50059    4.851908
4944     4.850535
4801     4.850535
Name: weighted_rating, dtype: float64

In [ ]:
# Assuming you want to display the 'name', 'location', and 'cuisine' columns as well
sorted_zomato = zomato.sort_values('weighted_rating', ascending=False).drop_duplicates(subset=['name'])
sorted_zomato.head(5)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost,reviews_list,menu_item,listed_type,listed_city,weighted_rating
49627,https://www.zomato.com/bangalore/byg-brewski-b...,"Behind MK Retail, Sarjapur Road, Bangalore",Byg Brewski Brewing Company,Yes,Yes,4.9,16832.0,+91 8039514766,Sarjapur Road,Microbrewery,"Cocktails, Butter Chicken, Dahi Kebab, Rajma C...","Continental, North Indian, Italian, South Indi...",1.6,"[('Rated 4.0', 'RATED\n Visiting a microbrewe...",[],Dine-out,Sarjapur Road,4.851908
37099,https://www.zomato.com/bangalore/abs-absolute-...,"100 Feet Road, 1st Phase, Near Jayadeva Flyove...",AB's - Absolute Barbecues,No,Yes,4.9,6490.0,040 45659912,BTM,Casual Dining,"Tangdi Chicken, Bbq Buffet, Chocolate Icecream...","European, Mediterranean, North Indian, BBQ",1.6,"[('Rated 5.0', 'RATED\n Thapush and Rahul ser...",[],Dine-out,Koramangala 7th Block,4.782763
41524,https://www.zomato.com/bangalore/the-black-pea...,"20/7, Swamy Legato, Outer Ring Road, Kadubeesa...",The Black Pearl,No,Yes,4.8,7284.0,080 49653069,Marathahalli,"Casual Dining, Bar","Dahipuri, Jal-jeera, Jalebi Rabri, Chicken Gri...","North Indian, European, Mediterranean, BBQ",1.5,"[('Rated 1.0', 'RATED\n Without any doubt thi...",[],Dine-out,Marathahalli,4.703220
47203,https://www.zomato.com/bangalore/chilis-americ...,"2nd Floor, Orion Mall, Dr. Rajkumar Road, Mall...",Chili's American Grill & Bar,Yes,Yes,4.8,6470.0,+91 9500075479\n+91 9535541956,Malleshwaram,"Casual Dining, Bar","Veg Platter, Burgers, Mushroom Soup, Cocktails...","American, Tex-Mex, Burger, BBQ, Mexican",1.4,"[('Rated 5.0', 'RATED\n I just love this plac...","['Bacon Ranch Chicken Quesadillas', 'Onion Rin...",Pubs and bars,Rajajinagar,4.692237
51042,https://www.zomato.com/bangalore/flechazo-whit...,"120 A3, 2nd Floor, Santosh Tower, EPIP Industr...",Flechazo,No,Yes,4.9,3004.0,+91 8884333312,Whitefield,Casual Dining,"Pizza, Mutton Kebab, Chicken Biryani, Sea Food...","Asian, Mediterranean, North Indian, BBQ",1.4,"[('Rated 4.0', ""RATED\n Last week I had a Tea...",[],Dine-out,Whitefield,4.672516


In [ ]:
zomato[['name','reviews_list']]

,name,reviews_list
0,Jalsa,"[('Rated 4.0', 'RATED\n A beautiful place to ..."
1,Spice Elephant,"[('Rated 4.0', 'RATED\n Had been here for din..."
2,San Churro Cafe,"[('Rated 3.0', ""RATED\n Ambience is not that ..."
3,Addhuri Udupi Bhojana,"[('Rated 4.0', ""RATED\n Great food and proper..."
4,Grand Village,"[('Rated 4.0', 'RATED\n Very good restaurant ..."
...,...,...
51712,Best Brews - Four Points by Sheraton Bengaluru...,"[('Rated 5.0', ""RATED\n Food and service are ..."
51713,Vinod Bar And Restaurant,[]
51714,Plunge - Sheraton Grand Bengaluru Whitefield H...,[]
51715,Chime - Sheraton Grand Bengaluru Whitefield Ho...,"[('Rated 4.0', 'RATED\n Nice and friendly pla..."


In [ ]:
zomato.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost', 'reviews_list', 'menu_item', 'listed_type',
       'listed_city', 'weighted_rating'],
      dtype='object')

## Candidates generation using label

In [ ]:
def match_and_filter(df, column, search_term, top_n=5):
    # Perform a precise match first
    precise_matches = df[df[column].astype(str).str.lower() == str(search_term).lower()]

    # If precise match results are fewer than top_n, perform fuzzy match
    if len(precise_matches) < top_n:
        matches = process.extract(search_term, df[column], scorer=process.scorer.WRatio, limit=None)
        matched_indices = [match[2] for match in matches if match[1] > 80]
        fuzzy_matches = df.iloc[matched_indices]
        combined_matches = pd.concat([precise_matches, fuzzy_matches]).drop_duplicates()
    else:
        combined_matches = precise_matches

    # Sort by weighed rating and remove duplicates by name
    top_items = combined_matches.sort_values(by='weighted_rating', ascending=False).drop_duplicates(subset='name').head(top_n)

    return top_items


## Candidates generation based on numerical information

In [ ]:
def filter_by_float_column(df, column, value, top_n=5):
    if isinstance(value, (int, float)):
        # Single number, filter within ±20%
        tolerance = 0.20 * value
        lower_bound = value - tolerance
        upper_bound = value + tolerance
        condition = df[column].between(lower_bound, upper_bound)
    elif isinstance(value, (list, tuple)) and len(value) == 2:
        # Two numbers, range filter
        lower_bound, upper_bound = sorted(value)  # Ensure lower to upper order
        condition = df[column].between(lower_bound, upper_bound)
    elif isinstance(value, dict):
        # One number with direction, e.g. {'bigger': 1000}
        number = next(iter(value.values()))
        direction = next(iter(value.keys()))
        if direction == 'bigger':
            condition = df[column] > number
        elif direction == 'smaller':
            condition = df[column] < number
        else:
            raise ValueError("Direction can only be 'bigger' or 'smaller'.")
    else:
        raise ValueError("Value must be a number, a range (list/tuple), or a dict with direction.")

    # Apply the filter
    filtered_df = df[condition]

    # Sort by 'weighed_rating' and remove duplicates by name
    top_items = filtered_df.sort_values(by='weighted_rating', ascending=False).drop_duplicates(subset='name').head(top_n)

    return top_items

##Merge and rank results

In [ ]:
def breakdown_and_collect(df, filter_list):
    scores = pd.Series(0, index=df.index)  # Initialize the scores series to zero for all indices

    for filter_dict in filter_list:
        for column, search_term in filter_dict.items():
            if isinstance(search_term, str):
                # Check for precise match in string columns and score 1 if matched
                scores[df[column].astype(str).str.lower() == search_term.lower()] += 1
            elif isinstance(search_term, (int, float)):
                # Check for match within range for float columns and assign appropriate score
                tolerance = search_term * 0.2
                upper_tolerance = search_term * 0.5
                within_tolerance = df[column].between(search_term - tolerance, search_term + tolerance)
                within_upper_tolerance = df[column].between(search_term - upper_tolerance, search_term + upper_tolerance)

                # Score 1 for values within 20% tolerance
                scores[within_tolerance] += 1
                # Score 0.5 for values that exceed 20% but are within 50% tolerance
                scores[(~within_tolerance) & within_upper_tolerance] += 0.5

    df['score'] = scores  # Assign scores to the dataframe
    df_matched = df[df['score'] > 0]  # Filter out rows with zero score

    # Sort by score and weighted_rating, and get top 5 unique restaurants
    df_matched = df_matched.sort_values(by=['score', 'weighted_rating'], ascending=[False, False])
    df_matched_unique = df_matched.drop_duplicates(subset='name').head(5)

    return df_matched_unique.drop(columns=['score'])  # Return results without the 'score' column



In [ ]:
# Example filter_list with unique columns
filter_list = [
    {'cuisines': 'Italian'},
    {'location': 'Banashankari'},
    {'approx_cost': 400},  # Exact value with 20% tolerance
]

# Assuming `zomato` is the DataFrame loaded with your data
final_results = breakdown_and_collect(zomato, filter_list)
print(final_results[['name', 'cuisines', 'approx_cost', 'location', 'weighted_rating']])

                         name  \
52     Corner House Ice Cream   
21364            Roving Feast   
3460       Mini Punjabi Dhaba   
145              Anna Kuteera   
13                 Caf-Eleven   

                                              cuisines  approx_cost  \
52                                 Ice Cream, Desserts        400.0   
21364  Italian, Continental, Fast Food, Chinese, Momos        450.0   
3460                                      North Indian        350.0   
145                South Indian, North Indian, Chinese        450.0   
13                                   Cafe, Continental        450.0   

           location  weighted_rating  
52     Banashankari         3.897820  
21364  Banashankari         3.879872  
3460   Banashankari         3.858381  
145    Banashankari         3.857134  
13     Banashankari         3.813146  


# create a classifation model

In [ ]:

# Create a copy of the list to avoid modifying the original data
df = pd.DataFrame(zomato.copy())


# Function to extract review text and rating from a review tuple
def extract_review(review_tuple):
  if len(review_tuple) == 1:
      # Handle tuples with a single element (e.g., assign empty string)
      return [""]
  else:
      rating, review = review_tuple
      return review.split('\n')[1:]

# Apply the extract_review function to each review in a new column
df['extracted_review'] = df['reviews_list'].apply(lambda x: [extract_review(review) for review in x])

# Flatten the extracted_review list using list comprehension
flat_reviews = [item for sublist in df['extracted_review'].tolist() for item in sublist]
df['flat_reviews'] = flat_reviews

# Create a new DataFrame resto_reviews containing name, extracted review, and rating
resto_reviews = df[['name', 'flat_reviews', rating := pd.to_numeric(df['reviews_list'].apply(lambda x: x[0][0].split()[1]))]]

# Print the resto_reviews DataFrame
print(resto_reviews.head())

In [ ]:
# First, ensure that the 'Rating' column is numeric and drop any non-numeric rows
resto_reviews['Rating'] = pd.to_numeric(resto_reviews['Rating'], errors='coerce')
resto_reviews = resto_reviews.dropna(subset=['Rating'])

# Next, remove rows with no rating (if any still remain)
resto_reviews = resto_reviews[resto_reviews['Rating'].notnull()]

# Now, apply the lambda function to create the 'Recommendation' column
resto_reviews['Recommendation'] = resto_reviews['Rating'].apply(lambda x: 'recommend' if x >= 3.0 else 'no recommend')

# The updated DataFrame now has a 'Recommendation' column with values based on the 'Rating' column


In [ ]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

resto_reviews["Review"] = resto_reviews["Review"].apply(lambda text: remove_stopwords(text))

## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

resto_reviews["Review"] = resto_reviews["Review"].apply(lambda text: remove_urls(text))